In [1]:
import os
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

### Load data 

In [2]:
#load data 
files = os.listdir('./data')  #list all the data files
dataDic = {}                  #use dictionary to store all the data
for i in range(len(files)):
    dataDic[files[i][:-4]] = pd.read_csv('./data/'+files[i]) 

scales_description_pd = dataDic['scales_description']  #185
participants_pd = dataDic['participants']          #185 
heart_rate_pd = dataDic['heart_rate']              #79   55
blood_pressure_pd = dataDic['blood_pressure']      #28    8
hrv_measurements_pd = dataDic['hrv_measurements']  #185  58
wearables_pd = dataDic['wearables']                #79   50
sleep_pd = dataDic['sleep']                        #10    7
surveys_pd = dataDic['surveys']                    #111  59 
weather_pd = dataDic['weather']                    #104  43

### Preprocess participants file 

In [3]:
#Preprocess participants file 
temp_par = participants_pd[['user_code','symptoms_onset','gender','age_range','country','height','weight']]
#If nan and invalid onset date, we use 01/01/2000 instead: 
Day1=[datetime.strptime(d,"%m/%d/%Y") if d==d and d[-4:]=='2020' else datetime.strptime('01/01/2000',"%m/%d/%Y") 
      for d in temp_par['symptoms_onset']]
temp_par.insert(2, "Day1", Day1)  #add a new column Day1 
temp_par = temp_par.drop(columns=['symptoms_onset']) #remove the old column 
temp_par['country'] = temp_par['country'].fillna('Unknown') # replace the nan country with Unknown 

### Preprocess heart_rate file

In [4]:
#Preprocess heart_rate file
temp_heart_rate = dataDic['heart_rate'].copy()          
Dates_hr = [datetime.strptime(temp_heart_rate['datetime'][i][:10],"%Y-%m-%d") for i in range(temp_heart_rate.shape[0])]
temp_heart_rate.insert(1,"Day", Dates_hr) 
#choose data with rest=0: 
temp_heart_rate = temp_heart_rate.loc[temp_heart_rate.is_resting==0,["user_code","Day","heart_rate"]] 
temp_heart_rate = temp_heart_rate.rename(columns={"heart_rate": "norest_heart_rate"}) 
temp_heart_rate = temp_heart_rate.groupby(['user_code','Day'],as_index=False).mean() #take averag

#Merge participants and heart_rate file 
merged_par_hr = pd.merge(temp_par, temp_heart_rate, how='left', on=['user_code'])
merged_par_hr["Day"] = list(i.days for i in merged_par_hr["Day"] - merged_par_hr["Day1"])  #convert Dat format 
#merged_par_hr #28.5% nan,  24.7% day<0 , 46.3% valid data 


### Preprocess hrv_measurements file 

In [5]:
#preprocess hrv_measurements file 
temp_hrv = dataDic['hrv_measurements'].copy()
Dates_hrv = [datetime.strptime(temp_hrv['measurement_datetime'][i][0:10],"%Y-%m-%d") for i in range(temp_hrv.shape[0])]
temp_hrv.insert(1,"Day", Dates_hrv) 
temp_hrv = temp_hrv[['user_code','Day','meanrr','sdnn','rmssd','lf','hf','vlf','lfhf','total_power']]
temp_hrv = temp_hrv.groupby(['user_code','Day'],as_index=False).mean() #take average for one day's record
 
#Merge participants and hrv_measurements file 
merged_par_hrv = pd.merge(temp_par, temp_hrv, how='left', on=['user_code'])
merged_par_hrv["Day"] = list(i.days for i in merged_par_hrv["Day"] - merged_par_hrv["Day1"])  
#merged_par_hrv  #20.7% nan,  21.8% day<0  #57.1% valid data 

### Preprocess wearables file 


In [6]:
#preprocess wearables file 
temp_wear = dataDic['wearables'].copy()
Dates_wear = [datetime.strptime(temp_wear['day'][i][0:10],"%Y-%m-%d") for i in range(temp_wear.shape[0])]
temp_wear.insert(1,"Day", Dates_wear) 
temp_wear = temp_wear[['user_code','Day','resting_pulse','pulse_average']]
temp_wear = temp_wear.groupby(['user_code','Day'],as_index=False).mean() 
temp_wear

#Merge participants and wearables file 
merged_par_wear = pd.merge(temp_par, temp_wear, how='left', on=['user_code'])
merged_par_wear["Day"] = list(i.days for i in merged_par_wear["Day"] - merged_par_wear["Day1"]) 
# 29.2% nan,  27.7% day<0,   1.62% valid data for resting_pulse, 31.1% valid data for pulse_average 

In [7]:
#test valid data 
#sum([( merged_par_wear['Day'][i].days <2000 and 
#      merged_par_wear['Day'][i].days>0 and 
#      merged_par_wear['Day'][i].days == merged_par_wear['Day'][i].days and
#      merged_par_wear['pulse_average'][i]==merged_par_wear['pulse_average'][i] ) 
#      for i in range(len(merged_par_wear['Day']))]  )/3204

### Preprocess surveys file 

In [8]:
#preprocess surveys file 
temp_sur = dataDic['surveys'].copy()
Dates_sur = [datetime.strptime(temp_sur['created_at'][i][0:10],"%Y-%m-%d") for i in range(temp_sur.shape[0])]
temp_sur.insert(1,"Day", Dates_sur) 
temp_sur = temp_sur[['user_code','Day','scale','value']]
temp_sur_x = temp_sur.loc[temp_sur.scale.isin(
    ['S_COVID_COUGH','S_COVID_FEVER','S_COVID_BREATH','S_COVID_FATIGUE','S_COVID_PAIN',
     'S_COVID_CONFUSION','S_COVID_TROUBLE','S_COVID_BLUISH']),['user_code','Day','scale','value']] 

duprows_x = [i for i in range(len(temp_sur_x)) if list(temp_sur_x[['user_code','Day','scale']].duplicated())[i]!=0]
temp_sur_x = temp_sur_x.drop(temp_sur_x.index[duprows_x]) #Remove the first duplicate records

#Merge participants and surveys file 
merged_par_sur = pd.merge(temp_par, temp_sur_x, how='left', on=['user_code'])
merged_par_sur["Day"] = list(i.days for i in merged_par_sur["Day"] - merged_par_sur["Day1"])
# 14.5% nan,  0.0% day<0,  85.4% valid data 
merged_par_sur = merged_par_sur.pivot_table(index=["user_code", "Day1","gender","age_range","country","height","weight","Day"], 
                           columns='scale', values='value').reset_index()  #long to wide, 203 rows 


### Merge participants and survey y

In [9]:
#Response variable(S_COVID_OVERALL)  
temp_sur_y = temp_sur.loc[temp_sur.scale.isin(['S_COVID_OVERALL']),['user_code','Day','scale','value']] 
duprows_y = [i for i in range(len(temp_sur_y)) if list(temp_sur_y[['user_code','Day','scale']].duplicated())[i]!=0]
temp_sur_y = temp_sur_y.drop(temp_sur_y.index[duprows_y]) #Remove the first duplicate records:
temp_sur_y = temp_sur_y.rename(columns={"value": "class6", 'scale':'scale_y'}) 

#Merge participants and Response variable   
merged_par_y = pd.merge(temp_par, temp_sur_y, how='left', on=['user_code'])
merged_par_y["Day"] = list(i.days for i in merged_par_y["Day"] - merged_par_y["Day1"])  
# 41.9% nan,  0.0% day<0,  58.1% valid data 
merged_par_y = merged_par_y.dropna(subset=['class6']) #remove nan, same effect as .dropna(how='any') 203 remain 

#add 3-class variable:
merged_par_y['class3'] = [1 if i in (1.,2.,3.) else 2 if i==4. else 3 for i in merged_par_y['class6']] 


### Merge survey x

In [10]:
#merge survey 
merged_par_y_sur = pd.merge(merged_par_y, merged_par_sur, how='left',
                            on=['user_code', 'Day1','gender','age_range','country','height','weight','Day'])
merged_par_y_sur = merged_par_y_sur.drop(columns=['scale_y']) #remove the old column 
unkDay1row = merged_par_y_sur[(merged_par_y_sur.Day1 == '2000-01-01')].index
merged_par_y_sur = merged_par_y_sur.drop(unkDay1row)

### Merge heart_rate and add sequence data 

In [11]:
#merge heart_rate 
merged_par_y_sur_hr = pd.merge(merged_par_y_sur, merged_par_hr, how='left',
                            on=['user_code', 'Day1','gender','age_range','country','height','weight','Day'])

norest_heart_rate_seq = [] #add heart rate sequence
for i in merged_par_y_sur_hr.index:
    hrseq = []
    pati = merged_par_y_sur_hr["user_code"][i]
    day = merged_par_y_sur_hr["Day"][i]
    for d in range(int(day)+1):
        hr = merged_par_hr[(merged_par_hr['user_code']==pati) & (merged_par_hr['Day']==d)]['norest_heart_rate'].values
        if len(hr) == 0:
            hrseq.append(0)  
        elif len(hr) == 1:
            hrseq.append(round(hr[0], 3))
    hrseq = ','.join(map(str, hrseq))    
    norest_heart_rate_seq.append(hrseq)      
merged_par_y_sur_hr['norest_heart_rate_seq'] = norest_heart_rate_seq


### Merge hrv_measurements and add sequence data 

In [12]:
#merge hrv_measurements 
merged_par_y_sur_hr_hrv = pd.merge(merged_par_y_sur_hr, merged_par_hrv, how='left',
                                   on=['user_code', 'Day1','gender','age_range','country','height','weight','Day'])

sdnn_seq = []   #add sdnn sequence
rmssd_seq =[]   #add rmssd sequence
lf_seq = []    #add lf sequence
hf_seq = []    #add hf sequence
vlf_seq = []   #add vlf sequence
lfhf_seq = []   #add lfhf sequence
total_power_seq = []   #add total_power sequence
meanrr_seq = []   #add meanrr sequence
var = ['sdnn','rmssd','lf','hf','vlf','lfhf','total_power','meanrr']

for i in merged_par_y_sur_hr_hrv.index:
    sdnnseq, rmssdseq, lfseq, hfseq, vlfseq, lfhfseq, tpseq, meanrrseq = [],[],[],[],[],[],[],[]
    var2seq = {'sdnn':sdnnseq, 'rmssd': rmssdseq, 'lf':lfseq, 'hf': hfseq,
               'vlf':vlfseq,   'lfhf': lfhfseq, 'total_power': tpseq, 'meanrr': meanrrseq}
    pati = merged_par_y_sur_hr_hrv["user_code"][i]
    day = merged_par_y_sur_hr_hrv["Day"][i]
    for v in var:
        for d in range(int(day)+1):
            point = merged_par_hrv[(merged_par_hrv['user_code']==pati) & (merged_par_hrv['Day']==d)][v].values
            if len(point) == 0:
                var2seq[v].append(0)  
            elif len(point) == 1:
                var2seq[v].append(round(point[0], 3)) 
            else:
                print('-----Error-----')
        var2seq[v] = ','.join(map(str, var2seq[v]))    
    sdnn_seq.append(var2seq['sdnn']) 
    rmssd_seq.append(var2seq['rmssd'])
    lf_seq.append(var2seq['lf'])
    hf_seq.append(var2seq['hf'])
    vlf_seq.append(var2seq['vlf'])
    lfhf_seq.append(var2seq['lfhf'])
    total_power_seq.append(var2seq['total_power'])
    meanrr_seq.append(var2seq['meanrr'])

merged_par_y_sur_hr_hrv['sdnn_seq'] = sdnn_seq
merged_par_y_sur_hr_hrv['rmssd_seq'] = rmssd_seq
merged_par_y_sur_hr_hrv['lf_seq'] = lf_seq
merged_par_y_sur_hr_hrv['hf_seq'] = hf_seq
merged_par_y_sur_hr_hrv['vlf_seq'] = vlf_seq
merged_par_y_sur_hr_hrv['lfhf_seq'] = lfhf_seq
merged_par_y_sur_hr_hrv['total_power_seq'] = total_power_seq
merged_par_y_sur_hr_hrv['meanrr_seq'] = meanrr_seq

### Final

In [13]:
merged_par_y_sur_hr_hrv.columns

Index(['user_code', 'Day1', 'gender', 'age_range', 'country', 'height',
       'weight', 'Day', 'class6', 'class3', 'S_COVID_BLUISH', 'S_COVID_BREATH',
       'S_COVID_CONFUSION', 'S_COVID_COUGH', 'S_COVID_FATIGUE',
       'S_COVID_FEVER', 'S_COVID_PAIN', 'S_COVID_TROUBLE', 'norest_heart_rate',
       'norest_heart_rate_seq', 'meanrr', 'sdnn', 'rmssd', 'lf', 'hf', 'vlf',
       'lfhf', 'total_power', 'sdnn_seq', 'rmssd_seq', 'lf_seq', 'hf_seq',
       'vlf_seq', 'lfhf_seq', 'total_power_seq', 'meanrr_seq'],
      dtype='object')

In [14]:
merged_par_y_sur_hr_hrv

,user_code,Day1,gender,age_range,country,height,weight,Day,class6,class3,...,lfhf,total_power,sdnn_seq,rmssd_seq,lf_seq,hf_seq,vlf_seq,lfhf_seq,total_power_seq,meanrr_seq
0,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,18.0,2.0,1,...,0.645250,283.750000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
1,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,20.0,2.0,1,...,1.564000,504.000000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
2,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,22.0,1.0,1,...,1.700333,404.333333,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
3,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,24.0,4.0,2,...,4.665000,967.250000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
4,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,28.0,5.0,3,...,1.782400,287.600000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
5,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,30.0,5.0,3,...,2.186000,650.000000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
6,01bad5a519,2020-04-05,m,45-54,Russia,178.00,92.000,38.0,6.0,3,...,6.315000,540.000000,"0,0,0,0,0,0,0,23.613,29.894,0,0,0,36.404,0,0,2...","0,0,0,0,0,0,0,28.472,28.847,0,0,0,26.684,0,0,2...","0,0,0,0,0,0,0,41.0,197.0,0,0,0,458.0,0,0,276.0...","0,0,0,0,0,0,0,61.0,268.0,0,0,0,70.5,0,0,63.0,0...","0,0,0,0,0,0,0,59.0,48.0,0,0,0,51.5,0,0,127.0,0...","0,0,0,0,0,0,0,0.672,0.735,0,0,0,7.63,0,0,4.381...","0,0,0,0,0,0,0,161.0,513.0,0,0,0,580.0,0,0,466....","0,0,0,0,0,0,0,714.17,880.56,0,0,0,787.185,0,0,..."
7,1ce1d77659,2020-04-04,m,25-34,United States,167.64,72.575,30.0,4.0,2,...,1.362500,1580.500000,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..

In [15]:
merged_par_y_sur_hr_hrv.to_csv(r'ProcessedData.csv', index = False)